# Webscraping for excel data from Sephora

The goal for this notebook is to provide usable excel data by pulling it from a site that utilizes lazy loading, and follows standard product container naming conventions. Each step is crucial to ensure data is aligned and matched properly to its related values.

The sections of this analysis include: 

 - Webdriver loader
     * Opens Sephora webpage
     * Scrapes through all product "divs" with specified classname, simulated scrolling to get past sites that utilize "lazy loading"
     * Computes whether data is already present, before deciding to add into main dataframe
     * Finalizes product data
 - Displaying partial data extracted.
 - Conversion of each column to an array, splicing them together.
 - Each of the products scraped is now parsed into a new webdriver, extracting the product ingredient's div container, specified by the class name.
 - Values present in this dataframe is then processed in a similar fashion to the products themselves.
 - Final result, an excel sheet containing the product name and ingredient data is output as an excel sheet.
 - Available data that can be added to the final sheet includes, 'Image URL', and 'URL Extension'.
 - In short, given a Sephora category url, this notebook outputs an excel sheet containing products present on that page, as well as its ingredients, showing its process every step of the way.

In [4]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup

product_names = set()
image_urls = set()
product_links = set()

driver = webdriver.Chrome()

time.sleep(5)

url = 'https://www.sephora.sg/brands/aveda/hair'
driver.get(url)



scroll_pause_time = 3
scroll_height = 0
while True:
    driver.execute_script(f"window.scrollTo(0, {scroll_height});")
    time.sleep(scroll_pause_time)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    containers = soup.find_all('div', {'class': 'products-card-container'})
    
    for container in containers:
        product_name = container.find('div', {'class': 'product-name'}).text.strip()
        product_link = 'https://www.sephora.sg' + container.find('a', {'class': 'product-card-image-link'})['href']
        product_image = container.find('img', {'class': 'product-card-image'})['src']
        if product_link not in product_links:
            product_names.add(product_name)
            image_urls.add(product_image)
            product_links.add(product_link)
    
    if len(product_links) >= 30:
        break
    scroll_height += 1000
    
driver.quit()

df = pd.DataFrame({'Product Name': list(product_names), 'Image Url': list(image_urls), 'Product Links': list(product_links)})




Displays a shortened version of the data extracted.
'Image URL' may not align with 'Product Name' currently.

In [5]:
print(df)

                                         Product Name  \
0            Nutriplenish™ Conditioner Light Moisture   
1                    Invati Advanced™ Thickening Foam   
2                 Botanical Repair Strengthening Trio   
3   Invati Advanced™ 3-Step System Light Set (Holi...   
4         Invati Advanced™  Exfoliating Shampoo Light   
5               Smooth Infusion™ Style-Prep™ Smoother   
6              Botanical Repair Strengthening Shampoo   
7                     Invati Advanced Light Hair Trio   
8                  Nutriplenish™ Leave-in Conditioner   
9   Botanical Repair Strengthening Leave-In Treatm...   
10     Invati Advanced™ Intensive Hair & Scalp Masque   
11       Nutriplenish™ Treatment Masque Deep Moisture   
12           Blonde Revival Purple Toning Conditioner   
13            Invati Advanced™ Thickening Conditioner   
14                   Nutriplenish Light Moisture Trio   
15            Pramāsana™ Protective Scalp Concentrate   
16               Rosemary Mint 



Develop 'Array' versions of each column. Data should match accurate at this stage.

In [6]:
dfArrayProduct = []
dfArrayImg = []
dfArrayLink = []
for container in containers:
        dfArrayProduct.append(container.find('div', {'class': 'product-name'}).text.strip())
        dfArrayImg.append(container.find('img', {'class': 'product-card-image'})['src'])
        dfArrayLink.append('https://www.sephora.sg' + container.find('a', {'class': 'product-card-image-link'})['href'])
print(dfArrayProduct)
print('')
print(dfArrayImg)
print('')
print(dfArrayLink)



['Invati Advanced™ Scalp Revitalizer', 'Invati Advanced™ Thickening Conditioner', 'Invati Advanced™  Exfoliating Shampoo Light', 'Botanical Repair Strengthening Leave-In Treatment For Hair', 'Invati Advanced™  Exfoliating Shampoo Rich', 'Rosemary Mint Purifying Shampoo', 'Invati Advanced™ Intensive Hair & Scalp Masque', 'Nutriplenish™ Shampoo Light Moisture', 'Smooth Infusion™ Style-Prep™ Smoother', 'Botanical Repair Strengthening Shampoo', 'Nutriplenish™ Daily Moisturizing Treatment', 'Shampure™ Nurturing Conditioner', 'Rosemary Mint Weightless Conditioner', 'Botanical Repair™ Strengthening Overnight Serum', 'Shampure™ Nurturing Shampoo', 'Invati Advanced™ Thickening Foam', 'Botanical Repair Strengthening Conditioner', 'Nutriplenish™ Treatment Masque Deep Moisture', 'Botanical Repair Intensive Strengthening Hair Masque Rich', 'Be Curly™ Curl Enhancer', 'Nutriplenish™ Multi-use Hair Oil', 'Be Curly™ Style Prep™', 'Nutriplenish™ Leave-in Conditioner', 'Botanical Repair Intensive Strengt

In [7]:
df = pd.DataFrame({'Product_Name': list(dfArrayProduct), 'Image_URL': list(dfArrayImg), 'Extensions': list(dfArrayLink)})

print(df)

                                         Product_Name  \
0                  Invati Advanced™ Scalp Revitalizer   
1             Invati Advanced™ Thickening Conditioner   
2         Invati Advanced™  Exfoliating Shampoo Light   
3   Botanical Repair Strengthening Leave-In Treatm...   
4          Invati Advanced™  Exfoliating Shampoo Rich   
5                     Rosemary Mint Purifying Shampoo   
6      Invati Advanced™ Intensive Hair & Scalp Masque   
7                Nutriplenish™ Shampoo Light Moisture   
8               Smooth Infusion™ Style-Prep™ Smoother   
9              Botanical Repair Strengthening Shampoo   
10         Nutriplenish™ Daily Moisturizing Treatment   
11                    Shampure™ Nurturing Conditioner   
12               Rosemary Mint Weightless Conditioner   
13    Botanical Repair™ Strengthening Overnight Serum   
14                        Shampure™ Nurturing Shampoo   
15                   Invati Advanced™ Thickening Foam   
16         Botanical Repair Str

In [8]:
print(dfArrayLink)

['https://www.sephora.sg/products/aveda-invati-advanced-scalp-revitalizer/v/150ml', 'https://www.sephora.sg/products/aveda-invati-advanced-thickening-conditioner/v/1l', 'https://www.sephora.sg/products/aveda-invati-advanced-exfoliating-shampoo-light/v/200ml', 'https://www.sephora.sg/products/aveda-botanical-repair-strengthening-leave-in-treatment-for-hair/v/100ml', 'https://www.sephora.sg/products/aveda-invati-advanced-exfoliating-shampoo-rich/v/200ml', 'https://www.sephora.sg/products/aveda-rosemary-mint-purifying-shampoo/v/250ml', 'https://www.sephora.sg/products/aveda-invati-advanced-intensive-hair-and-scalp-masque/v/150ml', 'https://www.sephora.sg/products/aveda-nutriplenish-shampoo-light-moisture/v/250ml', 'https://www.sephora.sg/products/aveda-smooth-infusion-style-prep-smoother/v/100ml', 'https://www.sephora.sg/products/aveda-botanical-repair-strengthening-shampoo/v/200ml', 'https://www.sephora.sg/products/aveda-nutriplenish-daily-moisturizing-treatment/v/150ml', 'https://www.se



Each product load will require 11s to ensure all data is loaded and extracted before the webdriver is closed.

In [9]:
print("Number of links: ",  str(len(dfArrayLink)))
print("Estimated Load Time (in seconds)", str(len(dfArrayLink) * 11))
for link in dfArrayLink:
    print(link)


Number of links:  34
Estimated Load Time (in seconds) 374
https://www.sephora.sg/products/aveda-invati-advanced-scalp-revitalizer/v/150ml
https://www.sephora.sg/products/aveda-invati-advanced-thickening-conditioner/v/1l
https://www.sephora.sg/products/aveda-invati-advanced-exfoliating-shampoo-light/v/200ml
https://www.sephora.sg/products/aveda-botanical-repair-strengthening-leave-in-treatment-for-hair/v/100ml
https://www.sephora.sg/products/aveda-invati-advanced-exfoliating-shampoo-rich/v/200ml
https://www.sephora.sg/products/aveda-rosemary-mint-purifying-shampoo/v/250ml
https://www.sephora.sg/products/aveda-invati-advanced-intensive-hair-and-scalp-masque/v/150ml
https://www.sephora.sg/products/aveda-nutriplenish-shampoo-light-moisture/v/250ml
https://www.sephora.sg/products/aveda-smooth-infusion-style-prep-smoother/v/100ml
https://www.sephora.sg/products/aveda-botanical-repair-strengthening-shampoo/v/200ml
https://www.sephora.sg/products/aveda-nutriplenish-daily-moisturizing-treatment

Ingredients webscraping from each product, utilizing the url extension of each product to find its relevant page & scraping ingredient data.

In [10]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

def get_product_ingredients(url):
    ingredientHeader = []
    service = Service('path/to/chromedriver') 
    driver = webdriver.Chrome(service=service)
    driver.get(url)
    try:
        WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CLASS_NAME, 'product-ingredients-values')))
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        ingredients_div = soup.find('div', {'class': 'product-ingredients-values'}).text.strip()
        header_div = soup.find('div', {'class' : "product-heading"}).h1.text
        ingredientHeader.append(ingredients_div)
        ingredientHeader.append(header_div)
        return ingredientHeader
    finally:
        driver.quit()

for url in dfArrayLink:
    ingredients = get_product_ingredients(url)
    print(ingredients[1])
    print(ingredients[0])
    print('')

Invati Advanced™ Scalp Revitalizer
Alcohol Denat., Water\Aqua\Eau, Glycerin, Citric Acid, Curcuma Longa (Turmeric) Root Extract, Panax Ginseng (Ginseng) Root Extract, Emblica Officinalis (Amla) Fruit Extract, Glycyrrhiza Glabra (Licorice) Root Extract, Citrus Reticulata (Tangerine) Peel Extract, Polygonum Cuspidatum Root (Japanese Knotweed) Extract, Camellia Sinensis (Green Tea) Leaf Extract, Laminaria Saccharina (Seaweed) Extract, Rubus Idaeus (Raspberry) Leaf Extract, Epilobium Fleisheri Extract, Chlorella Vulgaris Extract, Adenosine Phosphate, Arginine, Acetyl Carnitine Hcl, Creatine, Pyridoxine Hcl, Glucosamine Hcl, Caffeine, Tocopheryl Nicotinate, Tocopheryl Acetate, Malt Extract, Yeast Extract\Faex\Extrait De Levure, Panicum Miliaceum (Millet) Seed Oil, Hydroxypropyl Methylcellulose, Dipotassium Glycyrrhizate, Butylene Glycol, Peg-40 Hydrogenated Castor Oil, Hydroxypropylcellulose, Simethicone, Sodium Hydroxide, Fragrance (Parfum), Farnesol, Eugenol, Linalool, Benzyl Salicylate, 

In [26]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

def get_product_ingredients(url):
    service = Service('path/to/chromedriver')
    driver = webdriver.Chrome(service=service)
    driver.get(url)
    try:
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'product-ingredients-values')))
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        ingredients_div = soup.find('div', {'class': 'product-ingredients-values'}).text.strip()
        header_div = soup.find('div', {'class': "product-heading"}).h1.text
        return [header_div, ingredients_div]
    except:
        return []
    finally:
        driver.quit()

ingredients_list = [get_product_ingredients(url) for url in dfArrayLink]

df = pd.DataFrame(ingredients_list, columns=['product', 'ingredients'])

                                              product  \
0                Rosemary Mint Weightless Conditioner   
1                     Shampure™ Nurturing Conditioner   
2         Invati Advanced™  Exfoliating Shampoo Light   
3                         Shampure™ Nurturing Shampoo   
4                     Rosemary Mint Purifying Shampoo   
5             Invati Advanced™ Thickening Conditioner   
6                Nutriplenish™ Shampoo Light Moisture   
7                    Nutriplenish™ Multi-use Hair Oil   
8          Invati Advanced™  Exfoliating Shampoo Rich   
9                  Nutriplenish™ Leave-in Conditioner   
10         Nutriplenish™ Daily Moisturizing Treatment   
11              Smooth Infusion™ Style-Prep™ Smoother   
12       Nutriplenish™ Treatment Masque Deep Moisture   
13     Invati Advanced™ Intensive Hair & Scalp Masque   
14    Botanical Repair™ Strengthening Overnight Serum   
15                           Nutriplenish™ Curl Gelée   
16  Botanical Repair Strengthen

In [ ]:
print(df)



Generates excel sheet containing data shown in dataframe above.

In [11]:
df.to_excel('aveda-ingredients.xlsx', index=False, startrow=0)